# Instruction Finetuning

In this notebook, we will look into how to perform instruction finetuning. We will be doing full finetuning, i.e., retraining all the paramters of the model.

Load the required libraries

In [1]:
import os 
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


2.7.1+cu118
CUDA available: True
CUDA version: 11.8
Device name: NVIDIA GeForce RTX 3070 Laptop GPU


In [2]:
import os
#os.environ["WANDB_PROJECT"]="tinyllama_instruct_finetuning"

from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")


from enum import Enum
from functools import partial
import pandas as pd
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

d:\GENERATIVE_AI\XX.Finetuning_LLMs\mar_12_2025_revised_code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preprocessing: Creating Datasets and Dataloaders

In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1195k-token-2.5T"
dataset_name = "HuggingFaceH4/no_robots"
tokenizer = AutoTokenizer.from_pretrained(model_name)
template = """{% for message in messages %}\n{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% if loop.last and add_generation_prompt %}{{'<|im_start|>assistant\n' }}{% endif %}{% endfor %}"""
tokenizer.chat_template = template

In [4]:
def preprocess(samples):
    batch = []
    for conversation in samples["messages"]:
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    return {"content": batch}

dataset = load_dataset(dataset_name)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category'],
        num_rows: 9500
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category'],
        num_rows: 500
    })
})

In [6]:
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 9500
    })
    test: Dataset({
        features: ['content'],
        num_rows: 500
    })
})


In [7]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 9500
    })
    test: Dataset({
        features: ['content'],
        num_rows: 500
    })
})
{'content': '<|im_start|>user\nPlease summarize the goals for scientists in this text:\n\nWithin three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert’s portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat

In [8]:
dataset["test"][:10]

{'content': ['<|im_start|>system\nAster is a chatbot who answers questions with rhymes.<|im_end|>\n<|im_start|>user\nWhere did chocolate originate?<|im_end|>\n<|im_start|>assistant\nChocolate is 4000 years old/Mexico is where it was first sold<|im_end|>\n<|im_start|>user\nWhere was milk chocolate invented?<|im_end|>\n<|im_start|>assistant\nSwitzerland was the first to add milk/To make their chocolate smooth as silk<|im_end|>\n<|im_start|>user\nWhat are some good desserts that use chocolate?<|im_end|>\n<|im_start|>assistant\nPie, tart, cookies, and cake/Chocolate is great to bake<|im_end|>\n',
  "<|im_start|>user\nWhat are 5 things I can do when it's raining in London? I am visiting for the first time and only for a week with my husband. We love to walk, eat good food, and explore.<|im_end|>\n<|im_start|>assistant\nSure! Here are five options for things to do in London on a rainy day:\n\n1. Visit The British Museum. Dedicated to human history, art and culture, The British Museum has ove

## Loading the pretrained model and tokenizer

In [9]:
class ChatmlSpecialTokens(str, Enum):
    user = "<|im_start|>user"
    assistant = "<|im_start|>assistant"
    system = "<|im_start|>system"
    eos_token = "<|im_end|>"
    bos_token = "<s>"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        bos_token=ChatmlSpecialTokens.bos_token.value,
        eos_token=ChatmlSpecialTokens.eos_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
        trust_remote_code=True
    )
tokenizer.chat_template = template
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32005, 2048)

## Storing the base model predictions on a subset of 25 samples from eval test

In [10]:
tokenizer.padding_side="left"
def get_prediction_batched(samples, column_name):
    batch = []
    for conversation in samples["messages"]:
        chatml_gen_prompt = tokenizer.apply_chat_template(conversation[:-1], tokenize=False, add_generation_prompt=True)
        batch.append(chatml_gen_prompt)
    #text = tokenizer.apply_chat_template(conversation_history, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)#, add_special_tokens=False)
    inputs = {k: v.to("cuda") for k,v in inputs.items()}
    outputs = model.generate(**inputs, 
                             max_new_tokens=100, 
                             do_sample=True, 
                             top_p=0.95, 
                             temperature=0.2, 
                             repetition_penalty=1.1, 
                             eos_token_id=tokenizer.eos_token_id,
                             pad_token_id=tokenizer.eos_token_id,
                            )
    outputs = tokenizer.batch_decode(outputs)
    outputs = [output.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip() for output in outputs]
    return {column_name: outputs}


In [11]:
model.half().to("cuda")
test_dataset = load_dataset(dataset_name)["test"].shuffle().select(range(25))
test_dataset = test_dataset.map(
    partial(get_prediction_batched, column_name="base_assistant_message"),
    batched=True,
    batch_size=1)

print(test_dataset)
print(test_dataset[0])

d:\GENERATIVE_AI\XX.Finetuning_LLMs\mar_12_2025_revised_code\.venv\Lib\site-packages\torch\nn\modules\module.py:1341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
Map: 100%|██████████| 25/25 [01:18<00:00,  3.14s/ examples]

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'category', 'base_assistant_message'],
    num_rows: 25
})
{'prompt': "I just moved to New York State's Capital area. I am wondering where the Mohawk and Hudson Rivers come together. Where does that happen?", 'prompt_id': '9ea362fd9c2b04083756e2c03b02b4d5d334826313b7c36eb45f922068ccdb80', 'messages': [{'content': "I just moved to New York State's Capital area. I am wondering where the Mohawk and Hudson Rivers come together. Where does that happen?", 'role': 'user'}, {'content': 'The convergence of the Mohawk River, the former site of the Erie Canal, and the Hudson River happens a few miles north of Albany, New York by a town called Cohoes.', 'role': 'assistant'}], 'category': 'Open QA', 'base_assistant_message': 'Mohawk and Hudson Rivers come together at the confluence of the Hudson River and the Mohawk River in Albany, New York.\nThe Mohawk and Hudson Rivers meet at the confluence of the Hudson River and the Mohawk River in A

## Training

In [17]:
output_dir = "tinyllama_instruct"
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 8
logging_steps = 25
learning_rate = 2e-5
max_grad_norm = 1.0
max_steps = 250
num_train_epochs=1
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 128

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="no",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    optim="adamw_bnb_8bit",
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=False,
    bf16=True,
    report_to=["tensorboard"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)


d:\GENERATIVE_AI\XX.Finetuning_LLMs\mar_12_2025_revised_code\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from datasets import DatasetDict

# Assuming `dataset` is your DatasetDict
dataset = dataset.rename_columns({"content": "text"})

# Verify the change
print(dataset)

ValueError: Original column names {'content'} not in the dataset. Current columns in the dataset: ['text']

In [19]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    # packing=True,
    # dataset_text_field="content",
    # max_seq_length=max_seq_length,
)

C:\Users\Shariq\AppData\Local\Temp\ipykernel_22416\33760106.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Truncating eval dataset: 100%|██████████| 500/500 [00:00<00:00, 2384.73 examples/s]


In [20]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [21]:
trainer.train()
trainer.save_model()

Step,Training Loss
25,2.393000
50,2.198100
75,2.140100
100,2.115600
125,2.049800
150,2.032600
175,2.019400
200,2.018500
225,1.952400
250,1.962900


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]









model.safetensors:   0%|          | 16.4k/2.20G [00:00<11:17:44, 54.1kB/s]





model.safetensors:   0%|          | 459k/2.20G [00:00<25:16, 1.45MB/s]    
events.out.tfevents.1757758378.DARK-CYBERTON.18732.0: 100%|██████████| 10.9k/10.9k [00:01<00:00, 6.76kB/s]
events.out.tfevents.1757757801.DARK-CYBERTON.2000.2: 100%|██████████| 5.49k/5.49k [00:01<00:00, 3.41kB/s]
events.out.tfevents.1757757073.DARK-CYBERTON.2000.0: 100%|██████████| 5.50k/5.50k [00:01<00:00, 3.30kB/s]
events.out.tfevents.1757757568.DARK-CYBERTON.2000.1: 100%|██████████| 5.49k/5.49k [00:01<00:00, 3.15kB/s]







events.out.tfevents.1757758697.DARK-CYBERTON.18732.1: 100%|██████████| 5.49k/5.49k [00:00<00:00, 12.9kB/s]
events.out.tfevents.1757759087.DARK-CYBERTON.18732.4: 100%|██████████| 5.49k/5.49k [00:00<00:00, 11.8kB/s]
model.safetensors:   1%|          | 15.5M/2.20G [00:03<05:02, 7.23MB/s]



events.out.tfevents.1757759932.DARK-CYBERTON.18732.5

In [ ]:
!nvidia-smi

Sat Sep 13 15:55:20 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8    17W /  N/A |   7968MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading the trained model and getting the predictions of the trained model

In [31]:
#model = AutoModelForCausalLM.from_pretrained("shariq00/tinyllama_instruct", trust_remote_code=True)
model.to("cuda")
#model.to(torch.bfloat16)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32005, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [32]:
test_dataset = test_dataset.map(
    partial(get_prediction_batched, column_name="instruct_assistant_message"),
    batched=True,
    batch_size=1)

print(test_dataset)
print(test_dataset[0])

Map: 100%|██████████| 25/25 [01:03<00:00,  2.52s/ examples]

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'category', 'base_assistant_message', 'instruct_assistant_message'],
    num_rows: 25
})
{'prompt': "I just moved to New York State's Capital area. I am wondering where the Mohawk and Hudson Rivers come together. Where does that happen?", 'prompt_id': '9ea362fd9c2b04083756e2c03b02b4d5d334826313b7c36eb45f922068ccdb80', 'messages': [{'content': "I just moved to New York State's Capital area. I am wondering where the Mohawk and Hudson Rivers come together. Where does that happen?", 'role': 'user'}, {'content': 'The convergence of the Mohawk River, the former site of the Erie Canal, and the Hudson River happens a few miles north of Albany, New York by a town called Cohoes.', 'role': 'assistant'}], 'category': 'Open QA', 'base_assistant_message': 'Mohawk and Hudson Rivers come together at the confluence of the Hudson River and the Mohawk River in Albany, New York.\nThe Mohawk and Hudson Rivers meet at the confluence of the Hudson R

## Comparing the outputs of base model and instruction finetuned model

In [33]:
test_dataset = test_dataset.to_pandas()

In [34]:
pd.set_option("max_colwidth", 300)
test_dataset[["messages", "base_assistant_message", "instruct_assistant_message"]][:25]

,messages,base_assistant_message,instruct_assistant_message
0,"[{'content': 'I just moved to New York State's Capital area. I am wondering where the Mohawk and Hudson Rivers come together. Where does that happen?', 'role': 'user'}, {'content': 'The convergence of the Mohawk River, the former site of the Erie Canal, and the Hudson River happens a few miles n...","Mohawk and Hudson Rivers come together at the confluence of the Hudson River and the Mohawk River in Albany, New York.\nThe Mohawk and Hudson Rivers meet at the confluence of the Hudson River and the Mohawk River in Albany, New York. The river is a tributary of the Hudson River.\n\nHistory\nThe ...","The Mohawk and Hudson Rivers meet at the confluence of the two rivers in Albany, New York. The river is a major transportation route for goods and people throughout the region. It also provides water for drinking and irrigation."
1,"[{'content': 'Rewrite this text as if its part of a futuristic science fiction story: Tesla disabled the on-the-move passenger play feature via an over-the air software update after discussing the matter with officials. However, NHTSA continued its investigation. At the time, NHTSA reiterated t...",,"In 2018, Tesla disabled the on-the-move passenger play feature via an over-the air software update after discussing the matter with officials. However, NHTSA continued its investigation. At the time, NHTSA reiterated that the Vehicle Safety Act bars companies from selling cars that pose signific..."
2,"[{'content': 'Tell me how to get more Instagram followers please, as I want to become Instagram famous! Ten tips would be ideal. I deleted everything from my feed, so you can write it as if I'm starting from scratch! ', 'role': 'user'}, {'content': 'Good luck! Here are ten ways to get more Insta...",The first thing that I did was to delete all of the old posts and photos. I then went through and deleted any comments that were left by people who had commented on my old posts. I also deleted any photos that I had uploaded to my old account.\nI then took a look at my new account and found that...,"Here are some ways to get more Instagram followers:\n1. Create a catchy hashtag for your account and use it in all of your posts. This will help people find your account easily.\n2. Use hashtags that relate to the content of your post. For example, if you posted a photo of a dog, use #dog or #pe..."
3,"[{'content': 'Write a response to this email, stating that I did cite sources in the section on play therapy, and I did go over the strengths and weaknesses of play therapy. It seems she missed page two. Ask her to look over it again and regrade it. It should sound very professional. This was fo...",,"Dear Jane,\n\nThank you for completing this assignment. I appreciate your thoroughness and attention to detail. The brochure is well-written and clearly explains the services provided by Play Therapy. I also appreciated the references and citations you included.\n\nAs for the assessment, I notic..."
4,"[{'content': 'Please summarize this post. Make it easier to read and much shorter. It should only be about a paragraph or two. Use the text following this request. After World War II, designating the start and end dates for daylight saving time fell to state governments. Because this created m...",,"Daylight saving time has been a part of our lives since the late 1800s. The United States was one of the first countries to adopt this practice, but now, there are some states that want to change it. Some states have already voted to permanently switch to standard time, while others are consider..."
5,"[{'content': 'Write me a story outline for a funny story involving a man who is accidentally abducted by aliens. Write it as a series of 8 bullet points.', 'role': 'user'}, {'content': '* A man's car stops in the middle of nowhere. * He is beamed up by an alien spaceship. * The aliens seem confu...",1234567\nWrite a short story about a young boy who has to go to the hospita

In [35]:
messages = [
    {"role": "user", "content": "What an essay on Generative AI."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")#, add_special_tokens=False)
inputs = {k: v.to("cuda") for k,v in inputs.items()}
outputs = model.generate(**inputs, 
                         max_new_tokens=2000, 
                         do_sample=True, 
                         top_p=0.95, 
                         temperature=0.2, 
                         repetition_penalty=1.1, 
                         eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<s><|im_start|>user 
What an essay on Generative AI.<|im_end|> 
<|im_start|>assistant 
Generative AI is a type of artificial intelligence that can generate new ideas and concepts from scratch. It can be used to create new products, services, or even entire industries. Generative AI uses machine learning algorithms to learn patterns in data and then use those patterns to generate new ideas and concepts. This process can be very time-consuming and requires a lot of data to train the algorithm. However, generative AI has many benefits over traditional AI methods such as rule-based AI. For example, generative AI can produce more creative ideas than rule-based AI because it can learn from unstructured data like user feedback and user behavior. Additionally, generative AI can produce more accurate results than rule-based AI because it can take into account contextual information like user preferences and past experiences. Finally, generative AI can produce more efficient results than rule-ba

In [36]:
!nvidia-smi

Sun Sep 14 07:57:24 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P5    27W /  N/A |   7414MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------